In [1]:
import pandas as pd
import numpy as np
from matrixprofile import *
from matrixprofile.discords import discords
from matplotlib import pyplot as plt
from scipy.io import arff
from binarytree import Node
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import mutual_info_classif
from scipy.stats import entropy
from math import log, e
import pydotplus
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score

In [2]:
#VERSIONE CON USO DEI MOTIF,DISCORD O ENTRAMBI E CON MODULARITA' NELLA RIMOZIONE DEI CANDIDATI GIA SCELTI
# AGGIUNTA ANCHE LA FUNZIONE MIA PER SCEGLIERE IL BEST ATTRIBUTE E RELATIVO VALORE

In [45]:
def retrieve_all(Ts,window_size): #fornita la Ts calcola e restituisce mp, motifs, motifs_distances e discords
    Ts=Ts[0:127] #rimuovo l'attributo "classe"

    dfMP = pd.DataFrame(Ts).astype(float) # genero Dframe per lavorarci su, DA CAPIRE PERCHE SERVE FLOAT
    mp, mpi = matrixProfile.stomp(dfMP[0].values,window_size) #OK STOMP

    #PREPARO TUPLA DA PASSARE ALLA FUN MOTIF (RICHIEDE TUPLA FATTA DA MP E MPI)
    tupla=mp,mpi

    mot, motif_dist  = motifs.motifs(dfMP[0].values,tupla,2)

    #CALCOLO DISCORDS
    dis= discords(mp,window_size,2)
    #print('Discords starting position: '+str(dis))
    
    tupla=mp,mot,motif_dist,dis
    return tupla

In [44]:
#ogni motif e identificato da almeno due indici di partenza nella Ts, ne prendo uno solo rappresentativo
#genero poi struttura contenente gli indici di partenza di tutti i candidati

def candidateFilter(CandidatesList): 
    counterNumberMotif=0
    counterNumberDiscord=0
    l2=np.array([])
    for i in range (len(CandidatesList['Motif'])): #per ogni entry (per ogni record)
        numMotif=len(CandidatesList['Motif'].iloc[i])
        numDiscord=len(CandidatesList['Discord'].iloc[i])
        counterNumberDiscord+=numDiscord
        for j in range (numMotif): # per ogni lista di motif
            l1=CandidatesList['Motif'].iloc[i] #prima lista
            l2=np.append(l2,l1[j][0]) #prendo primo valore di ogni lista
            counterNumberMotif+=1
            
    
        CandidatesList['Motif'].iloc[i]=l2
        l2=np.array([]) #svuoto array
    
    return CandidatesList,counterNumberMotif,counterNumberDiscord

In [43]:
#riceve la lista di candidati e genera una lista della stessa dimensione di booleani
def buildCandidatesUsedList(CandidatesList,numberOfMotif,numberOfDiscord):
    CandidatesUsedList=pd.DataFrame(columns=['Used'],index=range(0,numberOfMotif+numberOfDiscord))
    boolList=[False] * (numberOfMotif+numberOfDiscord)
    CandidatesUsedList['Used']= boolList
    return CandidatesUsedList
                                    

In [35]:
def getDataStructures(df,window_size,verbose):
    #trasformo da stringa a numero il campo target 
    le = LabelEncoder()
    num_classes = le.fit_transform(df['target'])
    df['target']=num_classes
    df['TsIndex']=np.arange(len(df))
    window_size=5
    #diz={'Motif':[],'Motif-Dist':[],'Discord':[]}
    diz={'Motif':[],'Discord':[]}

    #CALCOLO MOTIF E DISCORD E LI INSERISCO NEL DIZIONARIO
    for i in range(len(df)):
        Ts = np.array(df.iloc[i][0:-2].values)
        mp,mot,motif_dist,dis = retrieve_all(Ts,window_size)
        diz['Motif'].insert(i, mot)
        diz['Discord'].insert(i, dis)

    #GENERO DFRAME DA DIZIONARIO

    CandidatesList = pd.DataFrame(diz)
    CandidatesList,numberOfMotif,numberOfDiscord=candidateFilter(CandidatesList)
    CandidatesUsedList=buildCandidatesUsedList(CandidatesList,numberOfMotif,numberOfDiscord)

    if(verbose==True):
        print('Candidati estratti')
        print(CandidatesList)
        print(numberOfMotif,numberOfDiscord)
        print(CandidatesUsedList)
    
    return mp,CandidatesList,numberOfMotif,numberOfDiscord,CandidatesUsedList

In [34]:
#per ogni Ts calcolo Dprofile con ogni candidato e inserisco la distanza minima con candidato i-esimo nella colonna i-esima
def computeSubSeqDistance(dataset,CandidatesList):
    
    #quantifico il num di candidati e in base a tale valore genero colonne per dfForDTree
    numberOfCandidates=0
    for i in range(len(CandidatesList)):
            numberOfCandidates+=len(CandidatesList['Motif'].loc[i])
            numberOfCandidates+=len(CandidatesList['Discord'].loc[i])
    columnsList=np.arange(numberOfCandidates)
    columnsList2=list()
    lastAttribute=['TsIndex','class']
    prefix='cand'
    for i in columnsList:
        columnsList2.append(prefix+str(i)) 
    columnsList2.append('TsIndex')
    columnsList2.append('class')
    dfForDTree=pd.DataFrame(columns=columnsList2,index=range(0,len(dataset)))

    #per ogni Ts, scandisco ogni candidato e calcolo la distanza minore 
    for i in range(len(dataset)):
        #acquisisco la Ts
        TsToCompare = np.array(dataset.iloc[i].values) 
        classValue=TsToCompare[128]
        TsToCompare=TsToCompare[0:128]
        dfForDTree['TsIndex'].iloc[i]=i
        dfForDTree['class'].iloc[i]=classValue
        counter=0
        #scandisco e calcolo distanza dai motif
        for j in range(len(CandidatesList)):
            numMotif=len(CandidatesList['Motif'].iloc[j])
            numDiscord=len(CandidatesList['Discord'].iloc[j])
            for k in range(numMotif):
                l1=CandidatesList['Motif'].iloc[j] #lista di indice i in motifDiscordList
                startingIndex=l1[k] #indice di inizio del motif
                TsContainingCandidateShapelet = np.array(dataset.iloc[j].values) #Ts contenente candidato shapelet
                Dp=distanceProfile.massDistanceProfile(TsContainingCandidateShapelet,int(startingIndex),window_size,TsToCompare)
                minValueFromDProfile=min(Dp[0]) #Dp[0] contiene il Dp effettivo
                dfForDTree[prefix+str(counter)].iloc[i]=minValueFromDProfile
                counter+=1
            for k in range(numDiscord):
                l1=CandidatesList['Discord'].iloc[j] #lista di indice i in motifDiscordList
                startingIndex=l1[k] #indice di inizio del motif
                TsContainingCandidateShapelet = np.array(dataset.iloc[j].values) #Ts contenente candidato shapelet
                Dp=distanceProfile.massDistanceProfile(TsContainingCandidateShapelet,int(startingIndex),window_size,TsToCompare)
                minValueFromDProfile=min(Dp[0]) #Dp[0] contiene il Dp effettivo
                dfForDTree[prefix+str(counter)].iloc[i]=minValueFromDProfile
                counter+=1
        
   # print(counter)    
    return dfForDTree #columnsList2 restituito per generare poi dFrame in "Split" (struttura dframe)
                
                

In [8]:
#dataset (dframe): nella riga i: indice della ts di appartenenza, distanza tra candidato e Ts, e classe di appartenenza di Ts
#calcola entropia di un dataset basandosi sul num di classi esistenti
def computeEntropy(dataset):
    value,counts = np.unique(dataset['class'], return_counts=True)
    actualEntropy=entropy(counts, base=2)
    return actualEntropy

In [42]:
#calcola il gain tra entropia nodo padre e sommatoria entropia nodi figli (GAIN CALCOLATO SUL VALORE DELL'ATTRIBUTO)
def computeGain(entropyParent,LenDatasetParent,Dleft,Dright):
    entropyLeft=computeEntropy(Dleft)
    entropyRight=computeEntropy(Dright)
    gain=entropyParent
    summation=( ((len(Dleft)/LenDatasetParent)*entropyLeft) +  ((len(Dright)/LenDatasetParent)*entropyRight) )
    gain=gain-summation
    return gain

In [10]:
#SPLIT SLAVE
#effettua lo split del dataset sul attributo e valore fornito
def split(dataset,attribute,value): 
    columnsList=dataset.columns.values
    dizLeft=pd.DataFrame(columns=columnsList)
    dizRight=pd.DataFrame(columns=columnsList)
    for i in range(len(dataset)):
        if dataset.iloc[i][attribute] < value:
            dizLeft = dizLeft.append(dataset.iloc[i], ignore_index=True)
        else:
            dizRight = dizRight.append(dataset.iloc[i], ignore_index=True)
    return dizLeft, dizRight

In [11]:
# riceve dframe con mutual_information(gain) e in base al candidatesGroup scelto, determina il miglior attributo su cui splittare
# che non è stato ancora utilizzato
def getBestIndexAttribute(vecMutualInfo,candidatesGroup,CandidatesUsedListTrain,numberOfMotif,numberOfDiscord):
        
    #ordino i candidati in base a gain decrescente
    
    vecMutualInfo=vecMutualInfo.sort_values(by='gain',ascending = False)
    
    #scandisco i candidati fino a trovare il candidato con miglior gain che non è ancora stato usato
    
    bestIndexAttribute=-1
    i=0
    
    #cicla fin quando trova candidato libero con gain maggiore
    while(bestIndexAttribute==-1 and i<len(vecMutualInfo)):    
        attributeToVerify=int(vecMutualInfo.iloc[i]['attribute'])
        if(CandidatesUsedListTrain.iloc[attributeToVerify]['Used']==False):
            bestIndexAttribute=attributeToVerify
            splitValue=vecMutualInfo.iloc[i]['splitValue']
            CandidatesUsedListTrain.iloc[attributeToVerify]=True #settando a true il candidato scelto, non sarà usato in seguito
        else:
            i+=1
    
    return bestIndexAttribute,splitValue
            
    
        
        

In [27]:
def computeMutualInfo(datasetForMutual,candidatesGroup,numberOfMotif,numberOfDiscord):
    #cerca attributo, il cui relativo best split value massimizza l'information gain nello split
    
    #definisco lista di indici inserire nella colonna 'attribute'
    if(candidatesGroup==0):
        candidatesIndex=range(numberOfMotif)
        numAttributes=numberOfMotif
    elif(candidatesGroup==1):
        candidatesIndex=range(numberOfMotif,numberOfMotif+numberOfDiscord)
        numAttributes=numberOfDiscord
    else:
        candidatesIndex=range(numberOfMotif+numberOfDiscord)
        numAttributes=numberOfMotif+numberOfDiscord
    
    columns=datasetForMutual.columns
    dframe=pd.DataFrame(columns=['attribute','splitValue','gain'],index=range(len(columns)-1)) #-1 cosi non prendo attr=class
    entropyParent=computeEntropy(datasetForMutual)

    
    
    #per ogni attributo, ordino il dframe sul suo valore
    #scandisco poi la y e appena cambia il valore di class effettuo uno split, memorizzando il best gain
    
    for i in range(len(columns)-1): #scandisco tutti gli attributi tranne 'class'
        bestGain=-1
        bestvalueForSplit=0
        previousClass=-1 #deve essere settato ad un valore non presente nei class value
        attribute=columns[i]
        datasetForMutual=datasetForMutual.sort_values(by=attribute,ascending = True)    
        
        y=datasetForMutual['class']
       
        
        for j in range(len(y)):
            if(j==0):
                previousClass=y[j]
                continue
            else:
                if(y[j]!=previousClass):
                    testValue=datasetForMutual.iloc[j][attribute]
                    Dleft,Dright=split(datasetForMutual,attribute,testValue)
                    actualGain=computeGain(entropyParent,len(datasetForMutual),Dleft,Dright)
                    if(actualGain > bestGain):
                        bestGain=actualGain
                        bestvalueForSplit=testValue
               
                previousClass=y[j] 
        # memorizzo in posizione i-esima lo split migliore e relativo gain
        
        dframe.iloc[i]['splitValue']=bestvalueForSplit
        dframe.iloc[i]['gain']=bestGain
        
    
    
    dframe['attribute']=candidatesIndex
    return dframe
        
        
    
    
    

In [40]:
#SPLIT INTERMEDIO
#dato il dataset, cerca il miglior attributo e relativo valore (optimal split point) su cui splittare
# restituiendo il dataset splittato e i valori trovati
def findBestAttributeValue(dataset,candidatesGroup,CandidatesUsedListTrain,numberOfMotif,numberOfDiscord,removeUsedCandidate,verbose):
    #VERBOSE AGGIUNGERE
    #cerca e restituisce attributo migliore su cui splittaree relativo valore ottimale (optimal split point)
    #CANDIDATE GROUP permette di scegliere se usare come candidati 0=motifs 1=discord 2=entrambi
    bestGain=0
    actualGain=0
    bestvalueForSplit=0
    y = dataset['class'].values
    y=y.astype('int')
    entropyParent=computeEntropy(dataset)
    
    #trovo best Attribute
    numAttributes=len(dataset.columns.values)
    numAttributes-=2 #tolgo i due attributi TsIndex e class dal Dframe
    datasetForMutual=pd.DataFrame()
    
    #preparo il Dframe da passare a mutual_info_classif, settando se scegliere tra motifs/discord/entrambi
    
    if(candidatesGroup==0): #solo motifs
        dataset=dataset.iloc[:,np.r_[:numberOfMotif]].copy()
    elif(candidatesGroup==1):
        datasetForMutual=dataset.iloc[:,np.r_[numberOfMotif:numberOfMotif+numberOfDiscord]].copy()
    else:
        datasetForMutual=dataset.iloc[:,np.r_[:numAttributes]].copy()

    datasetForMutual['class']=y
    
    #calcolo gain e miglior valore di split per ogni attributo
    
    
    vecMutualInfo=computeMutualInfo(datasetForMutual,candidatesGroup,numberOfMotif,numberOfDiscord)
    if(verbose==True):
        print('vec mutual info calcolato: ')
        print(vecMutualInfo)
    #se rimuovo candidati, faccio scegliere migliore non ancora utilizzato
    
    if(removeUsedCandidate==1): 
        indexBestAttribute,bestValueForSplit=getBestIndexAttribute(vecMutualInfo,candidatesGroup,CandidatesUsedListTrain,numberOfMotif,numberOfDiscord)
    else: #se non rimuovo candidati, mi basta prendere il primo
        vecMutualInfo=vecMutualInfo.sort_values(by='gain',ascending = False)
        indexBestAttribute=vecMutualInfo.iloc[0]['attribute']
        bestValueForSplit=vecMutualInfo.iloc[0]['splitValue']
    if(verbose==True):
        print('BEST attribute | value')
        print(indexBestAttribute,bestValueForSplit)
    
    splitValue=bestValueForSplit
    Dleft,Dright=split(dataset,indexBestAttribute,bestValueForSplit)
    
    
    return [indexBestAttribute,splitValue,Dleft,Dright]
    

In [41]:
#SPLIT MASTER
# funzione ricorsiva che implementa la creazione dell'albero di classificazione
# memorizza in ogni nodo: attributo, valore attributo su cui splitto, entropia nodo, num pattern
# memorizza in ogni foglia: entropia nodo, num pattern, classe nodo

# VERSIONE CHE RIMUOVE I CANDIDATI QUANDO VENGONO SCELTI 

def buildTree(actualNode,dataset,maxDepth, minSamplesLeaf, depth,candidatesGroup,CandidatesUsedListTrain,numberOfMotif,numberOfDiscord,removeUsedCandidate):
    #caso base: num pattern < soglia minima || profondità massima raggiunta => genero foglia con media delle classi
    #DATASET HA SEMPRE ALMENO UN PATTERN
    boolValue=checkIfIsLeaf(dataset)
    if(len(dataset)<minSamplesLeaf or depth>=maxDepth or boolValue==True ):
        average = sum(dataset['class'].values) / len(dataset['class'].values)
        classValue = round(average)
        numPattern=len(dataset)
        entropy=computeEntropy(dataset)
        
        nodeInfo=list()
        nodeInfo.append(classValue)
        nodeInfo.append(numPattern)
        nodeInfo.append(entropy)
    
        actualNode.data=nodeInfo
        actualNode.value=-1
        actualNode.left=None
        actualNode.right=None
        return 
    #caso ricorsivo in cui si può splittare
    else:
        
        returnList=findBestAttributeValue(dataset,candidatesGroup,CandidatesUsedListTrain,numberOfMotif,numberOfDiscord,removeUsedCandidate)
        indexChosenAttribute=returnList[0]
        attributeValue=returnList[1]
        Dleft=returnList[2]
        Dright=returnList[3]
        numPattern=len(dataset)
        entropy=computeEntropy(dataset)
        attributeList.append(indexChosenAttribute)
        
        #memorizzo nel nodo l'attributo, il valore e altre info ottenute dallo split
        
        nodeInfo=list()
        nodeInfo.append(attributeValue)
        nodeInfo.append(numPattern)
        nodeInfo.append(entropy)
        actualNode.data=nodeInfo
        actualNode.value=(indexChosenAttribute)
        
        #se possibile richiamo ricorsivamente sul nodo dx e sx figlio
        if(len(Dleft)>0):
            actualNode.left=Node(indexChosenAttribute)
            buildTree(actualNode.left,Dleft,maxDepth, minSamplesLeaf, depth+1,candidatesGroup,CandidatesUsedListTrain,numberOfMotif,numberOfDiscord,removeUsedCandidate)
        
        if(len(Dright)>0):
            actualNode.right=Node(indexChosenAttribute)
            buildTree(actualNode.right,Dright,maxDepth, minSamplesLeaf, depth+1,candidatesGroup,CandidatesUsedListTrain,numberOfMotif,numberOfDiscord,removeUsedCandidate)
            
        

In [15]:
#verifica se dataset, ha pattern appartenenti ad una sola classe => è gia foglia
def checkIfIsLeaf(dataset):
    isLeaf=True
    entropy=computeEntropy(dataset)
    if(entropy>0):
        isLeaf=False
    return isLeaf

In [39]:
#effettua il primo passo dell'algo di generazione dell'albero, richiama ricorsivamente sui figli
# VERSIONE CHE NON RIMUOVE I CANDIDATI QUANDO VENGONO SCELTI 
def startAlgo(dfForDTree,candidatesGroup,CandidatesUsedListTrain,maxDepth,minSamplesLeaf,numberOfMotif,numberOfDiscord,removeUsedCandidate,verbose):
    
    #inizio algo per nodo radice
    returnList=findBestAttributeValue(dfForDTree,candidatesGroup,CandidatesUsedListTrain,numberOfMotif,numberOfDiscord,removeUsedCandidate,verbose)
    indexChosenAttribute=returnList[0]
    attributeValue=returnList[1]
    Dleft=returnList[2]
    Dright=returnList[3]
    attributeList.append(indexChosenAttribute)
    root=Node(indexChosenAttribute)
    numPattern=len(dfForDTree)
    entropy=computeEntropy(dfForDTree)
        
    #memorizzo nel nodo l'attributo, il valore e altre info ottenute dallo split
        
    nodeInfo=list()
    nodeInfo.append(attributeValue)
    nodeInfo.append(numPattern)
    nodeInfo.append(entropy)
    root.data=nodeInfo
    
    root.left=Node(indexChosenAttribute)
    root.right=Node(indexChosenAttribute)
    
    #chiamata ricorsiva
    if(len(Dleft)>0):
        buildTree(root.left,Dleft,maxDepth, minSamplesLeaf,1,candidatesGroup,CandidatesUsedListTrain,numberOfMotif,numberOfDiscord,removeUsedCandidate)
    if(len(Dright)>0):
        buildTree(root.right,Dright,maxDepth, minSamplesLeaf, 1,candidatesGroup,CandidatesUsedListTrain,numberOfMotif,numberOfDiscord,removeUsedCandidate)
    return root

In [17]:
#stampa dell'albero
def printAll(Root):
    if(Root.left==None and Root.right==None):
        print('foglia')
    print('Nodo: '+str(Root.value))
    df=Root.data
    print(df)
    print("\n")
    if(Root.left!=None):
        printAll(Root.left)
    if(Root.right!=None):
        printAll(Root.right)

In [46]:
def predict(testDataset,root):
    #preparo dataset 
    numAttributes=len(testDataset.columns.values)
    numAttributes-=2 #per prendere solo gli attributi utili a xTest
    yTest=testDataset.iloc[:]['class'].values
    yPredicted=np.zeros( len(yTest) )
    xTest=testDataset.iloc[:,np.r_[:numAttributes]]
    
    #effettuo predizione per ogni pattern
    
    for i in range(len(xTest)):
        pattern=xTest.iloc[i]
        yPredicted[i]= treeExplorer(pattern,root)
    
    yTest = yTest.astype(int)
    yPredicted = yPredicted.astype(int)
    
    return yTest,yPredicted
        

In [47]:
def treeExplorer(pattern,node):
    #caso base, node è foglia
    if(node.value==-1):
        return int(node.data[0])
    else:
    #caso ricorsivo
        attr='cand'+str(node.value)
        if(pattern[attr] < node.data[0]):
            return treeExplorer(pattern,node.left)
        else:
            return treeExplorer(pattern,node.right)

In [20]:
def computeSubSeqDistanceForTest(datasetTest,datasetTrain,attributeList,CandidatesList,numberOfMotif,numberOfDiscord):
    #quantifico il num di candidati usati dall'albero e in base a tale valore genero colonne per dfForDTree
      #quantifico il num di candidati e in base a tale valore genero colonne per dfForDTree
    columnsList2=list()
    prefix='cand'
    for i in attributeList:
        columnsList2.append(prefix+str(i)) 
    columnsList2.append('TsIndex')
    columnsList2.append('class')
    dfForDTree=pd.DataFrame(columns=columnsList2,index=range(0,len(datasetTest)))

    #per ogni Ts, scandisco ogni candidato e calcolo la distanza minore 
    for i in range(len(datasetTest)):
        #acquisisco la Ts
        TsToCompare = np.array(datasetTest.iloc[i].values) 
        classValue=TsToCompare[128]
        TsToCompare=TsToCompare[0:128]
        dfForDTree['TsIndex'].iloc[i]=i
        dfForDTree['class'].iloc[i]=classValue
        counter=0
        #scandisco e calcolo distanza dai motif
        for z in range(len(attributeList)):
            candidateIndex=attributeList[z]
            counter=0
            for j in range(len(CandidatesList)):
                numMotif=len(CandidatesList['Motif'].iloc[j])
                numDiscord=len(CandidatesList['Discord'].iloc[j])
                for k in range(numMotif):
                    if(counter==candidateIndex):
                        l1=CandidatesList['Motif'].iloc[j] #lista di indice i in motifDiscordList
                        startingIndex=l1[k] #indice di inizio del motif
                        TsContainingCandidateShapelet = np.array(datasetTrain.iloc[j].values) #Ts contenente candidato shapelet
                        Dp=distanceProfile.massDistanceProfile(TsContainingCandidateShapelet,int(startingIndex),window_size,TsToCompare)
                        minValueFromDProfile=min(Dp[0]) #Dp[0] contiene il Dp effettivo
                        dfForDTree[prefix+str(counter)].iloc[i]=minValueFromDProfile
                    counter+=1
                        
                for k in range(numDiscord):
                    if(counter==candidateIndex):
                        l1=CandidatesList['Discord'].iloc[j] #lista di indice i in motifDiscordList
                        startingIndex=l1[k] #indice di inizio del motif
                        TsContainingCandidateShapelet = np.array(datasetTrain.iloc[j].values) #Ts contenente candidato shapelet
                        Dp=distanceProfile.massDistanceProfile(TsContainingCandidateShapelet,int(startingIndex),window_size,TsToCompare)
                        minValueFromDProfile=min(Dp[0]) #Dp[0] contiene il Dp effettivo
                        dfForDTree[prefix+str(counter)].iloc[i]=minValueFromDProfile
                    counter+=1
                    



    le = LabelEncoder()        
    num_classes = le.fit_transform(dfForDTree['class'])
    dfForDTree['class']=num_classes
            
    return dfForDTree #columnsList2 restituito per generare poi dFrame in "Split" (struttura dframe)
                

    

In [38]:
#ACQUISISCO STRUTTURE DATI DEL TRAINING SET
dataset = arff.loadarff('CBF/CBF_TRAIN.arff')
dfTrain=pd.DataFrame(dataset[0])
window_size=5
verbose=True
mpTrain,CandidatesListTrain,numberOfMotifTrain,numberOfDiscordTrain,CandidatesUsedListTrain=getDataStructures(dfTrain,window_size,verbose)
dfForDTree=computeSubSeqDistance(dfTrain,CandidatesListTrain)
if(verbose==True):
    print(dfForDTree)


Candidati estratti
            Motif    Discord
0     [9.0, 26.0]   [61, 27]
1   [105.0, 54.0]   [8, 119]
2    [17.0, 40.0]    [31, 1]
3    [48.0, 12.0]   [23, 86]
4          [27.0]   [38, 62]
5    [45.0, 13.0]  [119, 26]
6     [7.0, 37.0]   [98, 76]
7     [2.0, 24.0]    [36, 8]
8     [1.0, 29.0]   [23, 32]
9     [50.0, 0.0]   [38, 59]
10   [11.0, 17.0]   [80, 23]
11    [31.0, 7.0]  [114, 54]
12         [53.0]   [61, 10]
13    [79.0, 6.0]  [105, 38]
14         [59.0]    [98, 8]
15    [2.0, 45.0]     [1, 8]
16    [28.0, 3.0]    [5, 88]
17   [14.0, 35.0]  [118, 85]
18    [13.0, 3.0]   [111, 8]
19    [19.0, 6.0]   [92, 35]
20   [19.0, 39.0]   [26, 70]
21         [26.0]   [13, 35]
22   [18.0, 22.0]   [59, 25]
23   [27.0, 42.0]  [32, 104]
24    [8.0, 28.0]    [4, 95]
25         [16.0]   [87, 20]
26   [67.0, 59.0]   [33, 83]
27   [16.0, 39.0]   [75, 99]
28   [11.0, 41.0]   [47, 39]
29    [6.0, 15.0]  [117, 51]
55 60
      Used
0    False
1    False
2    False
3    False
4    False
..     ...

In [49]:
#COSTRUISCO DECISION TREE
candidatesGroup=1
albero=None
maxDepth=3
minSamplesLeaf=5
attributeList=list()
removeUsedCandidate=1
verbose=True
albero=startAlgo(dfForDTree[:5],candidatesGroup,CandidatesUsedListTrain,maxDepth,minSamplesLeaf,numberOfMotifTrain,numberOfDiscordTrain,removeUsedCandidate,verbose)
print(albero)
print(attributeList)
printAll(albero)




vec mutual info calcolato: 
    attribute splitValue      gain
0          55    0.98486  0.721928
1          56   0.691605  0.721928
2          57    0.90691  0.419973
3          58    0.65421  0.970951
4          59   0.718226  0.419973
5          60   0.556222  0.419973
6          61   0.983484  0.970951
7          62   0.920286  0.419973
8          63   0.770227  0.419973
9          64   0.671732  0.419973
10         65   0.952141  0.419973
11         66   0.599896  0.419973
12         67   0.859219  0.721928
13         68   0.809785  0.419973
14         69   0.917434  0.970951
15         70    1.02613  0.419973
16         71   0.740667  0.419973
17         72   0.850497  0.721928
18         73   0.759134  0.721928
19         74   0.984968  0.721928
20         75   0.630125  0.419973
21         76   0.797819  0.419973
22         77   0.628522  0.419973
23         78   0.732693  0.970951
24         79   0.955561  0.721928
25         80   0.787926  0.419973
26         81   0.933717  0

In [50]:
#GENERO STRUTTURE DATI PER TEST SET
window_size=5
verbose=True
dataset2 = arff.loadarff('CBF/CBF_TEST.arff')
dfTest = pd.DataFrame(dataset2[0]) #30 record su matrice da 128 attributi + 'b': classe appartenenza  
dfTest=dfTest.iloc[50:100] #ne prendo 50 altrimenti impiega tempo troppo lungo, sono 900 record totali

attributeList=sorted(attributeList)
dfForDTreeTest=computeSubSeqDistanceForTest(dfTest,dfTrain,attributeList,CandidatesListTrain,numberOfMotifTrain,numberOfDiscordTrain) 
if(verbose==True):
    print(dfForDTreeTest)



      cand58 TsIndex  class
0    1.24288       0      2
1   0.152125       1      0
2   0.898136       2      2
3   0.446341       3      0
4    0.37035       4      1
5    1.02427       5      0
6   0.730007       6      2
7   0.811578       7      2
8   0.278058       8      2
9   0.525072       9      2
10  0.927686      10      1
11  0.524187      11      0
12  0.294485      12      2
13   0.59094      13      0
14   0.54051      14      2
15  0.712961      15      0
16  0.873652      16      0
17  0.935544      17      2
18  0.711248      18      1
19  0.340839      19      0
20  0.610313      20      1
21  0.629947      21      2
22  0.821159      22      2
23  0.725304      23      0
24   0.95731      24      1
25  0.823237      25      1
26  0.823076      26      1
27  0.560534      27      1
28  0.509133      28      2
29  0.557955      29      1
30  0.884886      30      0
31  0.705443      31      0
32  0.923293      32      0
33  0.500157      33      0
34  0.891764      34

In [51]:
#EFFETTUO PREDIZIONE E MISURO RISULTATO

verbose=True
yTest,yPredicted=predict(dfForDTreeTest,albero)


if(verbose==True):
    for a,b in zip(yTest,yPredicted):
        print(a,b)
    
print(type(yPredicted))
print(type(yTest))
    
print(classification_report(yTest, yPredicted))
print('Accuracy %s' % accuracy_score(yTest, yPredicted))
print('F1-score %s' % f1_score(yTest, yPredicted, average=None))
confusion_matrix(yTest, yPredicted)    

2 2
0 0
2 2
0 0
1 0
0 2
2 2
2 2
2 0
2 0
1 2
0 0
2 0
0 0
2 0
0 2
0 2
2 2
1 2
0 0
1 0
2 0
2 2
0 2
1 2
1 2
1 2
1 0
2 0
1 0
0 2
0 2
0 2
0 0
2 2
2 0
2 2
0 2
2 2
0 2
2 2
0 2
1 2
1 0
0 0
1 2
2 2
2 2
0 2
2 0
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
              precision    recall  f1-score   support

           0       0.35      0.39      0.37        18
           1       0.00      0.00      0.00        12
           2       0.40      0.60      0.48        20

    accuracy                           0.38        50
   macro avg       0.25      0.33      0.28        50
weighted avg       0.29      0.38      0.32        50

Accuracy 0.38
F1-score [0.36842105 0.         0.48      ]


C:\Users\Allen\softwaretesi1\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


array([[ 7,  0, 11],
       [ 5,  0,  7],
       [ 8,  0, 12]], dtype=int64)